In [ ]:
# Goal: temporally composite one tile for one period.

#(also see how long it takes)

In [1]:
import numpy as np
import os
import rasterio

In [12]:
from scipy.interpolate import CubicSpline

In [2]:
def filter_granule(granule_path,
                        mark_if_clouds=True,
                        mark_if_shadow=True,
                        mark_if_adjacent=True,
                        mark_if_snow=False):

    granule = granule_path[-34:]
    
    fmask_tiff = f'{granule}.Fmask.tif'
    fmask_path = f'{granule_path}/{fmask_tiff}'

    try:
        with rasterio.open(fmask_path) as src:
            fmask_arr = src.read(1).flatten()
    except Exception:
        print('An fmask file could not be read:', fmask_path)
        print('Marking that granule as bad data.')
        # If we can't read the fmask, we need to ditch this granule
        fin = [np.full((3660*3660,), -9999,
                       dtype='int16') for _ in range(7)]
        return fin

    marked = []
    for el in fmask_arr:
        binary = '{0:b}'.format(el).zfill(8)
        has_clouds = bool(int(binary[-2]))
        has_adjacent = bool(int(binary[-3]))
        has_shadow = bool(int(binary[-4]))
        mark = ((has_clouds and mark_if_clouds) or
              (has_adjacent and mark_if_adjacent) or
              (has_shadow and mark_if_shadow))
        marked.append(mark)    
    cas_mask = np.array(marked)
    
    ### SNOW PROCESSING
    if mark_if_snow:
        band2 = 'B02'
        tiff2 = f'{granule}.{band2}.tif'
        path2 = f'{granule_path}/{tiff2}'
        try:
            with rasterio.open(path2) as src2:
                arr2 = src2.read(1).flatten()
        except Exception:
            print('A file could not be read:', path2)
            print('Cannot determine snow, so we cannot use this granule any further.')
            print('Marking that granule as bad data.')
            fin = [np.full((3660*3660,), -9999,
                       dtype='int16') for _ in range(7)]
            return fin
        snow_mask = arr2>2000
        final_mask = snow_mask | cas_mask
    else:
        final_mask = cas_mask
    
    ### For the 6 color bands,
    ### Read into arrays and flatten
    L30_dict = {'Blue':'B02', 'Green':'B03', 'Red':'B04',
               'NIR':'B05', 'SWIR1':'B06', 'SWIR2':'B07'}
    S30_dict = {'Blue':'B02', 'Green':'B03', 'Red':'B04',
               'NIR':'B8A', 'SWIR1':'B11', 'SWIR2':'B12'}
    bands = ['Blue','Green','Red','NIR','SWIR1','SWIR2']
    DOY = int(granule[-15:-12])
    sat = granule[4:7]
    
    filtered_granule = []
    
    for band in bands:
        if sat == 'L30':
            band_code = L30_dict[band]
        if sat == 'S30':
            band_code = S30_dict[band]
        
        band_tiff = f'{granule}.{band_code}.tif'
        band_path = f'{granule_path}/{band_tiff}'

        try:
            with rasterio.open(band_path) as src:
                band_arr = src.read(1).astype('int16').flatten()
        except Exception:
            print('A band file could not be read:', band_path)
            print('Marking that as bad data and continuing as normal with the rest of the files.')
            # If we can't read this color band, mark it as all -9999
            band_arr = np.full(final_mask.shape, -9999, dtype='int16')
        
        filtered_granule.append(band_arr)

    filtered_granule.append(final_mask)
    
    ## Note: if we wanted to we could also return DOY and sat. Dunno if they will be helpful.
    
    ### return filtered_+granule as a list of 7 flat arrays
    ## the 7 arrays are: blue, greem, red, NIR, SWIR1, SWIR2, mask
    return filtered_granule 

### Try running that function!

In [3]:
def list_granule_paths(tile='10SFH',
                       year=2020,
                       hls_dir='../data/hls_23feb23',
                       sats=['L30','S30'],
                       period_start=1,
                       period_end=366):
    
    tile_slashes = f'{tile[:2]}/{tile[2]}/{tile[3]}/{tile[4]}'
    
    granule_paths = []
    
    for sat in sats:
        path = f'{hls_dir}/{sat}/{year}/{tile_slashes}'
        these_granules = os.listdir(path)
        
        this_sat_paths = []
        for granule in these_granules:
            DOY = int(granule[-15:-12])
            if DOY>=period_start and DOY<=period_end:
                this_sat_paths.append(path+'/'+granule)
            
        granule_paths += this_sat_paths
    
    return granule_paths

In [4]:
granule_paths = list_granule_paths(tile='10SFH',
                       year=2020,
                       hls_dir='../data/hls_23feb23',
                       sats=['L30','S30'])

### Sweet, filtering that one granule took about 10 seconds!

In [ ]:
granules_in_period = list_granule_paths(year=2020,
                                       tile='10SFH',
                                       period_start=91,
                                       period_end=104)

In [ ]:
len(granules_in_period)

In [ ]:
filtered_granule

In [ ]:
list_of_lists = [[] for _ in range(7)]
print(list_of_lists)
list_of_lists[0].append(88)
list_of_lists[1].append(55)
list_of_lists[0].append(12)

In [ ]:
list_of_lists[0]

In [ ]:
list_of_lists[1]

In [ ]:
list_of_lists[2]

In [ ]:
## Fill list_of_lists with all the data to be composited
list_of_lists = [[] for _ in range(7)]
for granule_path in granules_in_period:
    # Filter the granule
    filtered_granule = filter_granule(granule_path)
  
    # Record all the bands and the mask
    for i in range(7):
        list_of_lists[i].append(filtered_granule[i])

# Now list_of_lists has all the data we want!

Note: defining list_of_lists as `[[]] * 7` will not work. If we do it that way, we have 7 views of hte same list: appending to one of the views will therefore append to the one shared list and be visible in all 7 views. 

Old interrogation of that below...

What is in the mysterious list_of_lists?

Conclusion: Within each lis, maybe none of the elements are the same.
Next wuestion: is the first el of list 0 the same as the first element of list 1? Yes.

In [ ]:
for i, lis in enumerate(list_of_lists):
    print(f'-- Info about list {i} --')
    print(f'Length of list {i} is {len(lis)}')
    print('Are some of the elements the same as each other? /n',
          lis[0]==lis[4], lis[1]==lis[5], lis[2]==lis[6])
    print('placeholder')



In [ ]:
for i in range(6):
    for j in range(28):
        print(list_of_lists[i][j].all()==list_of_lists[i+1][j].all())


In [ ]:
len(list_of_lists[2])

In [ ]:
len(list_of_lists)

In [ ]:
len(list_of_lists[0][0])

In [ ]:
list_of_lists

In [ ]:
# Now take the median of each color band
fmask_filter = np.array(list_of_lists[-1])
composited = []
for i in range(6):
    marr = np.ma.array(list_of_lists[i])
    marr.mask = (marr==-9999) | fmask_filter
    this_band = np.ma.median(marr, axis=0).astype('int16')
    composited.append(this_band.filled(-9999))
    
# Now composited contains the median-composited data for all bands
# for just this period, and for just this tile.

#### Question:
What happends if we try to take the median of four things that are masked?

Answer (proof in other notebook): We get a masked value, where the data is 0. That is, the data value is 0 and the mask value is True.

In [ ]:
composited.data

## Next steps

- put all lof the above into one function
- run that giant function for all periods

In [4]:
def composite_one_period(tile='10SFH',
                        year=2020,
                        period_start=91,
                        period_end=104,
                        hls_dir='../data/hls_23feb23',
                        sats=['L30','S30'],
                        mark_if_clouds=True,
                        mark_if_shadow=True,
                        mark_if_adjacent=True,
                        mark_if_snow=False):

    granules_in_period = list_granule_paths(period_start=period_start,
                                           period_end=period_end,
                                           year=year,
                                           tile=tile,
                                           hls_dir=hls_dir,
                                           sats=sats)

    ## Fill list_of_lists with all the data to be composited
    list_of_lists = [[] for _ in range(7)]
    for granule_path in granules_in_period:
        # Filter the granule
        filtered_granule = filter_granule(granule_path,
                                          mark_if_clouds=mark_if_clouds,
                                          mark_if_shadow=mark_if_shadow,
                                          mark_if_adjacent=mark_if_adjacent,
                                          mark_if_snow=mark_if_snow)

        # Record all the bands and the mask
        for i in range(7):
            list_of_lists[i].append(filtered_granule[i])

    # Now take the median of each color band
    fmask_filter = np.array(list_of_lists[-1])
    composited = []
    for i in range(6):
        marr = np.ma.array(list_of_lists[i])
        marr.mask = (marr==-9999) | fmask_filter
        this_band = np.ma.median(marr, axis=0).astype('int16')
        composited.append(this_band.filled(-9999))

    # Now composited contains the median-composited data for all bands
    # for just this period, and for just this tile.
    return composited

In [ ]:
composited = composite_one_period(period_start=105,
                    period_end=118)

In [ ]:
composited

In [5]:
# Define periods
def get_period_schemes():

    first_pd = (1,90)
    last_pd = (301,366)
    meat_5day = [(i,i+4) for i in range(91,300,5)]
    meat_14day = [(i,i+13) for i in range(91,300,14)]

    scheme_5day = [first_pd] + meat_5day + [last_pd]
    scheme_14day = [first_pd] + meat_14day + [last_pd]
    
    return scheme_5day, scheme_14day

In [6]:
scheme_5day, scheme_14day = get_period_schemes()

In [7]:
def composite_tile_year(tile,
                            year,
                            period_scheme,
                            hls_dir='../data/hls_23feb23',
                            sats=['L30','S30'],
                            mark_if_clouds=True,
                            mark_if_shadow=True,
                            mark_if_adjacent=True,
                            mark_if_snow=False):

    feature_names = []
    features = []

    for period in period_scheme:
        
        period_start = period[0]
        period_end = period[1]
        
        # NAMES
        name_ohne_band = f'Refl_{period_start}_{period_end}'
        bands = ['Blue','Green','Red','NIR','SWIR1','SWIR2']
        names_this_period = [f'{name_ohne_band}_{band}' for band in bands]
        feature_names += names_this_period
        
        # DATA
        composited_this_period = composite_one_period(tile=tile,
                            year=year,
                            period_start=period_start,
                            period_end=period_end,
                            hls_dir=hls_dir,
                            sats=sats,
                            mark_if_clouds=mark_if_clouds,
                            mark_if_shadow=mark_if_shadow,
                            mark_if_adjacent=mark_if_adjacent,
                            mark_if_snow=mark_if_snow)
        features += composited_this_period
        
        print(f'Finished period ending DOY {period_end}.')
        
        # features is a list of flat numpy arrays
        # feature_names is a list of strings
        
    return features, feature_names

In [8]:
composited_10SFH_2020 = composite_tile_year(tile='10SFH',
                   year=2020,
                   period_scheme=scheme_14day)

Finished period ending DOY 90.
Finished period ending DOY 104.
Finished period ending DOY 118.
Finished period ending DOY 132.
Finished period ending DOY 146.
Finished period ending DOY 160.
Finished period ending DOY 174.
Finished period ending DOY 188.
Finished period ending DOY 202.
Finished period ending DOY 216.
Finished period ending DOY 230.
Finished period ending DOY 244.
Finished period ending DOY 258.
Finished period ending DOY 272.
Finished period ending DOY 286.
Finished period ending DOY 300.
Finished period ending DOY 366.


In [9]:
features, feature_names = composited_10SFH_2020

### Next: write functions for *interpolating* and *exploring how many periods needed interpolating*

In [10]:
n_pixels = len(features[0])
n_features = len(features)
array_new = np.full(shape=(n_pixels,n_features),
                    fill_value=-9999,dtype='int16')

In [13]:
for i_pixel in range(n_pixels): # for every pixel in the 14 million
    if i_pixel%1000000 == 0:
        print(f'Sit tight! Fitting cubic splines on the {i_pixel}th pixel...')
    for band in range(6): # for each band in the 6
        #range with skip # gather the data for this pixel this band
        these_data = [features[j][i_pixel] for j in range(band, n_features, 6)]
        ### these_data gets gathered from `features`
        ### these_names get gathered from `feature_names`
        meat = these_data[1:-1]

        t = [t for t,refl in enumerate(meat) if refl!=-9999]
        r = [refl for refl in meat if refl!=-9999]

        f = CubicSpline(t,r)

        t_new = np.arange(len(meat))
        r_new = f(t_new).astype('int16')
        r_sandwich = np.concatenate(([these_data[0]],
                                    r_new,
                                    [these_data[-1]]),
                                    dtype='int16')
        
        # Above: fit a cubic spline
        # get new refl values
        
        # Below: write new refls back into `features`
        ##### just establish a new 2d Numpy array filled with -9999
        ### this is where it *might* be more convenient to write this to a numpy array
        ### could even be a 3-d Numpy array...
        ### each output of the inner part of this for loop will be
        ### a specific pixel-band, but all periods
        ### nah, no 3d numpy array
        # write these_names into final_names.. or do the data writing in such
        # a way that the features are in the same order (using range with skip)
        for k,j in enumerate(range(band, n_features, 6)):
        # plunk each element of r_new out there into array_new
            array_new[i_pixel,j] = r_sandwich[k]

Sit tight! Fitting cubic splines on the 0th pixel...
Sit tight! Fitting cubic splines on the 1000000th pixel...
Sit tight! Fitting cubic splines on the 2000000th pixel...
Sit tight! Fitting cubic splines on the 3000000th pixel...
Sit tight! Fitting cubic splines on the 4000000th pixel...
Sit tight! Fitting cubic splines on the 5000000th pixel...
Sit tight! Fitting cubic splines on the 6000000th pixel...
Sit tight! Fitting cubic splines on the 7000000th pixel...
Sit tight! Fitting cubic splines on the 8000000th pixel...
Sit tight! Fitting cubic splines on the 9000000th pixel...
Sit tight! Fitting cubic splines on the 10000000th pixel...
Sit tight! Fitting cubic splines on the 11000000th pixel...
Sit tight! Fitting cubic splines on the 12000000th pixel...
Sit tight! Fitting cubic splines on the 13000000th pixel...


In [59]:
array_new.nbytes

2732702400

In [14]:
np.save('../data/Refl_10SFH_2020_14day.npy', array_new, allow_pickle=False)

In [58]:
array_new[12055400]

array([ 306,  648,  467, 3114, 2220, 1190,  260,  576,  387, 3302, 1651,
        809,  396,  758,  930, 3128, 2536, 1333,  553,  933, 1523, 3128,
       3100, 1667,  542,  784, 1069, 1609, 2585, 1962,  517,  742, 1007,
       1534, 2669, 1984,  581,  807, 1059, 1693, 2905, 2100,  637,  873,
       1174, 1838, 3104, 2222,  258,  336,  448,  683, 1423, 1597,  287,
        382,  507,  763, 1568, 1752,  331,  440,  578,  867, 1720, 1875,
        372,  506,  659,  983, 1852, 1891,  396,  531,  704, 1045, 1956,
       1999,  411,  516,  701, 1125, 2200, 2192,  487,  617,  817, 1239,
       2351, 2356,  540,  679,  907, 1335, 2452, 2398,  542,  699,  936,
       1457, 2507, 2402], dtype=int16)

In [49]:
array_new.any() == -9999

False

In [50]:
array_new.shape

(13395600, 102)

In [42]:
len(array_new[0,:])

102

## Gather the features of a particular band for cubic spline interpolation

Recall that features rotate predictably: blue, green, red, nir, swir1, swir2, blue, green, red, nir, ....

Therefore we know that features of index in range(0, end, 6) are all the blue features. And featuyres of index in range(1, end, 6) are all the green features. (2, end, 6) for red features. Etc. 

And we'd like to grab only the 1th feature through the second-to-last feature. How do we do this? ---> might be easiest to just grab all the red features, then when we're fitting the cubic spline only input features in the slice 1:-1.

In [ ]:
#Gather red features
#red_features = [features[i] for i in range(2, len(features), 6)]

red_features_one_pixel = [features[i][0] for i in range(2, len(features), 6)]

In [ ]:
# Do cubic spline on the 0th pixel, time series 1:-1


In [16]:
from scipy.interpolate import CubicSpline

In [ ]:
import numpy as np

In [ ]:
red_features_one_pixel = [300,400,350,-9999,330,410]

In [ ]:
meat = red_features_one_pixel[1:-1]

t = [t for t,refl in enumerate(meat) if refl!=-9999]
r = [refl for refl in meat if refl!=-9999]

f = CubicSpline(t,r)

t_new = np.arange(len(meat))

meat

t_new

r_new = f(t_new).astype('int16')

In [ ]:
r_new

In [ ]:
## Write r_new into the official dataset
# something here

In [ ]:
x = [0, 1, 2]
y = [1, 3, 2]

# use bc_type = 'natural' adds the constraints as we described above
f = CubicSpline(x, y, bc_type='natural')
x_new = np.linspace(0, 2, 100)
y_new = f(x_new)

In [ ]:
x = [period[0] for period in period_scheme[1:-1]]

In [ ]:
y = refls_this_band

In [ ]:
f = CubicSpline(x, y)



We're just going to have to run CubicSpline() 14 million times. Oof.


In [ ]:
for i_pixel in range(len(features[0])): # for every pixel in the 14 million
    for j_band in range(6): # for each band in the 6
        #range with skip # gather the data for this pixel this band
        ### these_data gets gatehred from `features`
        ### these_names get gathered from `feature_names`
        # fit a cubic spline
        # get new refl values
        # write new refls back into `features`
        ##### just establish a new 2d Numpy array filled with -9999
        ### this is where it *might* be more convenient to write this to a numpy array
        ### could even be a 3-d Numpy array...
        ### each output of the inner part of this for loop will be
        ### a specific pixel-band, but all periods
        ### nah, no 3d numpy array
        # write these_names into final_names.. or do the data writing in such
        # a way that the features are in the same order (using range with skip)

In [ ]:
r_sandwich = np.concatenate(([12],r_new,[16]),dtype='int16')

In [ ]:
r_sandwich

In [ ]:
20001 % 1000

In [ ]:
np.arange(8).append(16)

In [18]:
these_data = np.array([200,300,400,500,600])

In [20]:
these_data.shape

(5,)

In [22]:
these_data[1:-1].shape

(3,)

In [26]:
[these_data[0]]

[200]

In [28]:
r_sandwich = np.concatenate(([these_data[0]],
                                    these_data[1:-1],
                                    [these_data[-1]]),
                                    dtype='int16')

In [30]:
r_sandwich

array([200, 300, 400, 500, 600], dtype=int16)

In [1]:
import numpy as np
import os
import rasterio

from scipy.interpolate import CubicSpline

def filter_granule(granule_path,
                        mark_if_clouds=True,
                        mark_if_shadow=True,
                        mark_if_adjacent=True,
                        mark_if_snow=False):

    granule = granule_path[-34:]
    
    fmask_tiff = f'{granule}.Fmask.tif'
    fmask_path = f'{granule_path}/{fmask_tiff}'

    try:
        with rasterio.open(fmask_path) as src:
            fmask_arr = src.read(1).flatten()
    except Exception:
        print('An fmask file could not be read:', fmask_path)
        print('Marking that granule as bad data.')
        # If we can't read the fmask, we need to ditch this granule
        fin = [np.full((3660*3660,), -9999,
                       dtype='int16') for _ in range(7)]
        return fin

    marked = []
    for el in fmask_arr:
        binary = '{0:b}'.format(el).zfill(8)
        has_clouds = bool(int(binary[-2]))
        has_adjacent = bool(int(binary[-3]))
        has_shadow = bool(int(binary[-4]))
        mark = ((has_clouds and mark_if_clouds) or
              (has_adjacent and mark_if_adjacent) or
              (has_shadow and mark_if_shadow))
        marked.append(mark)    
    cas_mask = np.array(marked)
    
    ### SNOW PROCESSING
    if mark_if_snow:
        band2 = 'B02'
        tiff2 = f'{granule}.{band2}.tif'
        path2 = f'{granule_path}/{tiff2}'
        try:
            with rasterio.open(path2) as src2:
                arr2 = src2.read(1).flatten()
        except Exception:
            print('A file could not be read:', path2)
            print('Cannot determine snow, so we cannot use this granule any further.')
            print('Marking that granule as bad data.')
            fin = [np.full((3660*3660,), -9999,
                       dtype='int16') for _ in range(7)]
            return fin
        snow_mask = arr2>2000
        final_mask = snow_mask | cas_mask
    else:
        final_mask = cas_mask
    
    ### For the 6 color bands,
    ### Read into arrays and flatten
    L30_dict = {'Blue':'B02', 'Green':'B03', 'Red':'B04',
               'NIR':'B05', 'SWIR1':'B06', 'SWIR2':'B07'}
    S30_dict = {'Blue':'B02', 'Green':'B03', 'Red':'B04',
               'NIR':'B8A', 'SWIR1':'B11', 'SWIR2':'B12'}
    bands = ['Blue','Green','Red','NIR','SWIR1','SWIR2']
    DOY = int(granule[-15:-12])
    sat = granule[4:7]
    
    filtered_granule = []
    
    for band in bands:
        if sat == 'L30':
            band_code = L30_dict[band]
        if sat == 'S30':
            band_code = S30_dict[band]
        
        band_tiff = f'{granule}.{band_code}.tif'
        band_path = f'{granule_path}/{band_tiff}'

        try:
            with rasterio.open(band_path) as src:
                band_arr = src.read(1).astype('int16').flatten()
        except Exception:
            print('A band file could not be read:', band_path)
            print('Marking that as bad data and continuing as normal with the rest of the files.')
            # If we can't read this color band, mark it as all -9999
            band_arr = np.full(final_mask.shape, -9999, dtype='int16')
        
        filtered_granule.append(band_arr)

    filtered_granule.append(final_mask)
    
    # return filtered_granule as a list of 7 flat arrays
    # the 7 arrays are: blue, greem, red, NIR, SWIR1, SWIR2, mask
    return filtered_granule 

def list_granule_paths(tile='10SFH',
                       year=2020,
                       hls_dir='../data/hls_23feb23',
                       sats=['L30','S30'],
                       period_start=1,
                       period_end=366):
    
    tile_slashes = f'{tile[:2]}/{tile[2]}/{tile[3]}/{tile[4]}'
    
    granule_paths = []
    
    for sat in sats:
        path = f'{hls_dir}/{sat}/{year}/{tile_slashes}'
        these_granules = os.listdir(path)
        
        this_sat_paths = []
        for granule in these_granules:
            DOY = int(granule[-15:-12])
            if DOY>=period_start and DOY<=period_end:
                this_sat_paths.append(path+'/'+granule)
            
        granule_paths += this_sat_paths
    
    return granule_paths

def composite_one_period(tile='10SFH',
                        year=2020,
                        period_start=91,
                        period_end=104,
                        hls_dir='../data/hls_23feb23',
                        sats=['L30','S30'],
                        mark_if_clouds=True,
                        mark_if_shadow=True,
                        mark_if_adjacent=True,
                        mark_if_snow=False):

    granules_in_period = list_granule_paths(period_start=period_start,
                                           period_end=period_end,
                                           year=year,
                                           tile=tile,
                                           hls_dir=hls_dir,
                                           sats=sats)

    ## Fill list_of_lists with all the data to be composited
    list_of_lists = [[] for _ in range(7)]
    for granule_path in granules_in_period:
        # Filter the granule
        filtered_granule = filter_granule(granule_path,
                                          mark_if_clouds=mark_if_clouds,
                                          mark_if_shadow=mark_if_shadow,
                                          mark_if_adjacent=mark_if_adjacent,
                                          mark_if_snow=mark_if_snow)

        # Record all the bands and the mask
        for i in range(7):
            list_of_lists[i].append(filtered_granule[i])

    # Now take the median of each color band
    fmask_filter = np.array(list_of_lists[-1])
    composited = []
    for i in range(6):
        marr = np.ma.array(list_of_lists[i])
        marr.mask = (marr==-9999) | fmask_filter
        this_band = np.ma.median(marr, axis=0).astype('int16')
        composited.append(this_band.filled(-9999))

    # Now composited contains the median-composited data for all bands
    # for just this period, and for just this tile.
    return composited

def get_period_schemes():

    first_pd = (1,90)
    last_pd = (301,366)
    meat_5day = [(i,i+4) for i in range(91,300,5)]
    meat_14day = [(i,i+13) for i in range(91,300,14)]

    scheme_5day = [first_pd] + meat_5day + [last_pd]
    scheme_14day = [first_pd] + meat_14day + [last_pd]
    
    return scheme_5day, scheme_14day


def composite_tile_year(tile,
                            year,
                            period_scheme,
                            hls_dir='../data/hls_23feb23',
                            sats=['L30','S30'],
                            mark_if_clouds=True,
                            mark_if_shadow=True,
                            mark_if_adjacent=True,
                            mark_if_snow=False):

    feature_names = []
    features = []

    for period in period_scheme:
        
        period_start = period[0]
        period_end = period[1]
        
        # NAMES
        name_ohne_band = f'Refl_{period_start}_{period_end}'
        bands = ['Blue','Green','Red','NIR','SWIR1','SWIR2']
        names_this_period = [f'{name_ohne_band}_{band}' for band in bands]
        feature_names += names_this_period
        
        # DATA
        composited_this_period = composite_one_period(tile=tile,
                            year=year,
                            period_start=period_start,
                            period_end=period_end,
                            hls_dir=hls_dir,
                            sats=sats,
                            mark_if_clouds=mark_if_clouds,
                            mark_if_shadow=mark_if_shadow,
                            mark_if_adjacent=mark_if_adjacent,
                            mark_if_snow=mark_if_snow)
        features += composited_this_period
        
        print(f'Finished period ending DOY {period_end}.')
        
        # features is a list of flat numpy arrays
        # feature_names is a list of strings
        
    return features, feature_names


def interpolate_cubic_spline(features):

    n_pixels = len(features[0])
    n_features = len(features)
    array_new = np.full(shape=(n_pixels,n_features),
                        fill_value=-9999,dtype='int16')

    for i_pixel in range(n_pixels): # for every pixel in the 14 million
        if i_pixel%1000000 == 0:
            print(f'Sit tight! Fitting cubic splines on the {i_pixel}th pixel...')
        for band in range(6): # for each band in the 6
            # gather the data for this pixel this band
            these_data = [features[j][i_pixel] for j in range(band, n_features, 6)]
            
            meat = these_data[1:-1]

            t = [t for t,refl in enumerate(meat) if refl!=-9999]
            r = [refl for refl in meat if refl!=-9999]

            f = CubicSpline(t,r)

            t_new = np.arange(len(meat))
            r_new = f(t_new).astype('int16')
            r_sandwich = np.concatenate(([these_data[0]],
                                        r_new,
                                        [these_data[-1]]),
                                        dtype='int16')

            # write the filled features into array_new in the same order
            for k,j in enumerate(range(band, n_features, 6)):
            # distribute each element of r_new out into array_new
                array_new[i_pixel,j] = r_sandwich[k]

